An example Scholia / Wikidata / WikiCite profile for "environmental justice" is here
https://scholia.toolforge.org/topic/Q1479527

for the resource hub, we may want ["Recently published works on the topic"]([text](https://scholia.toolforge.org/topic/Q1479527#recently-published-works)), as with https://scholia.toolforge.org/topic/Q1479527#recently-published-works

The SPARQL query for that is
https://github.com/WDscholia/scholia/blob/master/scholia/app/templates/topic_recently-published-works.sparql

In [ ]:
query = '''
PREFIX target: <http://www.wikidata.org/entity/Q1479527>

SELECT ?date ?work ?workLabel (CONCAT("/work/", SUBSTR(STR(?work), 32)) AS ?workUrl)
  ?topicsUrl ?topics 
WITH {
  SELECT DISTINCT ?work WHERE {
    ?work wdt:P921 / (wdt:P361+ | wdt:P1269+ | (wdt:P31* / wdt:P279*) ) target: .
  }
} AS %works
WITH {
  SELECT (MAX(?dates) as ?datetime) ?work (GROUP_CONCAT(DISTINCT ?topic_label; separator=" // ") AS ?topics) 
                                    (CONCAT("../topics/", GROUP_CONCAT(DISTINCT SUBSTR(STR(?topic), 32); separator=",")) AS ?topicsUrl) 
  WHERE {
    INCLUDE %works
    ?work wdt:P921 ?topic . 
    OPTIONAL { ?work wdt:P577 ?dates . }
    ?topic rdfs:label ?topic_label .  FILTER (lang(?topic_label) = 'en')
  }
  GROUP BY ?work
} AS %result
WHERE {
  INCLUDE %result

  # There is a problem with BC dates
  # BIND(xsd:date(?datetime) AS ?date)
  BIND(REPLACE(STR(?datetime), 'T.*', '') AS ?date)
    
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh". }
}
GROUP BY ?date ?work ?workLabel ?topicsUrl ?topics
ORDER BY DESC(?date)
LIMIT 500
'''

In [ ]:
import python.data_extraction as DEX

data_extracter = DEX.WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
df.head()